In this notebook I test deploying my tflite trained models as a REST API.  

Source:  
https://www.kdnuggets.com/2021/04/deploy-machine-learning-models-to-web.html

Conda env `tf2_gpu_berta_api`

#### create a REST API using FAST API

##### Create an `app.py` file:
```
import numpy as np
from fastapi import FastAPI, Form, File, UploadFile
import pandas as pd
from starlette.responses import HTMLResponse
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import re

### Create an input form so that the "users" could input data
### In this case should be an image

app = FastAPI()

@app.get('/predict', response_class=HTMLResponse)
def take_inp():
    return '''
        <form method="post">
        <input type="file" id="file" placeholder="upload a file" name="file" />
        <input type="submit" />'''
        
        
@app.post('/predict')
def predict(file:UploadFile = File(...)):

    interpreter = tf.lite.Interpreter(model_path='model4.tflite')
    interpreter.allocate_tensors()
    
    # Load the input shape required by the model
    _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
    
    # Load the input image and preprocess it
    preprocessed_image, original_image = preprocess_image(image_path,(input_height, input_width))
    
    img = tf.io.decode_image(file, channels=3)
    img = tf.image.convert_image_dtype(img, tf.uint8)
    original_image = img
    resized_img = tf.image.resize(img, input_size)
    resized_img = resized_img[tf.newaxis, :]
    resized_img = tf.cast(resized_img, dtype=tf.uint8)

    # predict:
    signature_fn = interpreter.get_signature_runner()

    # Feed the input image to the model
    output = signature_fn(images=image)
    
    boxes = np.squeeze(output['output_3'])
    
    return boxes
    
```

In [ ]:
!uvicorn app:app --reload

INFO:     Will watch for changes in these directories: ['/home/ella/Desktop/berta_match_routes']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [27714] using StatReload
2022-09-22 00:53:30.527565: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-22 00:53:30.527618: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
INFO:     Started server process [27716]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     127.0.0.1:37612 - "GET /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:47026 - "POST /predict HTTP/1.1" 422 Unprocessable Entity
